In [1]:
import pickle
X_hr,y_stress,y_participant,X_time = pickle.load(open('../data/tabular_data_60_seconds_ppg_rr.p','rb'))

X_hr = X_hr.reshape(-1,30,1)

import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,LeaveOneGroupOut,LeavePGroupsOut
from sklearn.metrics import accuracy_score
from tensorflow import keras
from tensorflow.keras import backend as K 
import matplotlib.pyplot as plt
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
import numpy as np
%matplotlib notebook
from keras.layers import Conv1D,Reshape,BatchNormalization,TimeDistributed, \
Dropout,Input,MaxPooling1D,Flatten,Dense,Input, GaussianNoise,LSTM, Bidirectional, Input
from keras.models import Model, Sequential
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer,LabelEncoder
from sklearn.metrics.pairwise import euclidean_distances

train_participant = y_participant.copy()

print(np.unique(y_participant),train_participant.shape)

n_classes = len(np.unique(y_participant))
n_classes

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14] (42928,)


15

In [2]:
train_x = X_hr
train_y = y_participant

train_x, test_x, train_y, test_y,participant_ids_train, participant_ids_test = train_test_split(X_hr,
                                                            train_participant,
                                                            y_participant,
                                                            test_size = 0.5,
                                                            random_state=41,
                                                            stratify=y_participant)
train_x, val_x, train_y, val_y, participant_ids_train, participant_ids_val  = train_test_split(train_x,
                                                            train_y,
                                                            participant_ids_train,
                                                            test_size = 0.1,
                                                            random_state=41,
                                                            stratify=participant_ids_train)
train_x.shape,test_x.shape,val_x.shape,train_y.shape,test_y.shape,val_y.shape,participant_ids_train.shape

def get_model(input_shape=(30,1),act='relu',loss="sparse_categorical_crossentropy",opt='adam',n_classes=350):
    model =  Sequential()
    model.add(Input(shape=input_shape))
    model.add(Reshape(input_shape))
    model.add(Conv1D(100,10,input_shape=input_shape,activation='linear',kernel_initializer='normal',padding='same'))
    model.add(Conv1D(100,10,input_shape=input_shape,activation='relu',kernel_initializer='normal',padding='same'))
    model.add(MaxPooling1D(2))
    model.add(BatchNormalization())
    model.add(Dropout(.2))
    model.add(Conv1D(200,10,activation='relu',kernel_initializer='normal',padding='same'))
    model.add(MaxPooling1D(2))
    model.add(BatchNormalization())
    model.add(Dropout(.2))
    model.add(Conv1D(300,10,activation='relu',kernel_initializer='normal',padding='same'))
    model.add(MaxPooling1D(2))
    model.add(BatchNormalization())
    model.add(Conv1D(50,2,activation='relu',kernel_initializer='normal',padding='same'))
    model.add(MaxPooling1D(2))
    model.add(BatchNormalization())
    model.add(Dropout(.2))
    model.add(Flatten())
    model.add(Dense(n_classes*2,activation=None))
    model.add(Dense(n_classes,activation=None))
    model.add(tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1)))
    model.add(Dense(n_classes,activation='softmax',kernel_initializer='normal'))
    model.compile(loss=loss,optimizer=opt,metrics=['acc'])
    return model

model = get_model(input_shape=(30,1),n_classes=n_classes)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 30, 1)             0         
_________________________________________________________________
conv1d (Conv1D)              (None, 30, 100)           1100      
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 30, 100)           100100    
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 15, 100)           0         
_________________________________________________________________
batch_normalization (BatchNo (None, 15, 100)           400       
_________________________________________________________________
dropout (Dropout)            (None, 15, 100)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 15, 200)           2

In [3]:
from keras.models import load_model
filepath = '../model_files/base_cnn_60_seconds_ppg_hr_wesad_ig.hdf5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=40)
callbacks_list = [es,checkpoint]
history = model.fit(train_x,train_y,validation_data=(val_x,val_y), epochs=400, batch_size=200,
          callbacks=callbacks_list,shuffle=True)

model = load_model(filepath)
train_embeddings = model.predict(train_x)

Epoch 1/400
97/97 [==============================] - ETA: 0s - loss: 2.6210 - acc: 0.1666
Epoch 00001: val_loss improved from inf to 2.70024, saving model to ../model_files/base_cnn_60_seconds_ppg_hr_wesad_ig.hdf5
97/97 [==============================] - 1s 15ms/step - loss: 2.6210 - acc: 0.1666 - val_loss: 2.7002 - val_acc: 0.0894
Epoch 2/400
97/97 [==============================] - ETA: 0s - loss: 2.5161 - acc: 0.1966
Epoch 00002: val_loss improved from 2.70024 to 2.58338, saving model to ../model_files/base_cnn_60_seconds_ppg_hr_wesad_ig.hdf5
97/97 [==============================] - 1s 11ms/step - loss: 2.5161 - acc: 0.1966 - val_loss: 2.5834 - val_acc: 0.1658
Epoch 3/400
94/97 [============================>.] - ETA: 0s - loss: 2.4584 - acc: 0.2068
Epoch 00003: val_loss improved from 2.58338 to 2.45395, saving model to ../model_files/base_cnn_60_seconds_ppg_hr_wesad_ig.hdf5
97/97 [==============================] - 1s 11ms/step - loss: 2.4580 - acc: 0.2071 - val_loss: 2.4539 - val_ac

97/97 [==============================] - 1s 11ms/step - loss: 1.5919 - acc: 0.4562 - val_loss: 1.7712 - val_acc: 0.3987
Epoch 29/400
94/97 [============================>.] - ETA: 0s - loss: 1.5389 - acc: 0.4777
Epoch 00029: val_loss improved from 1.77119 to 1.72165, saving model to ../model_files/base_cnn_60_seconds_ppg_hr_wesad_ig.hdf5
97/97 [==============================] - 1s 12ms/step - loss: 1.5407 - acc: 0.4766 - val_loss: 1.7217 - val_acc: 0.3982
Epoch 30/400
92/97 [===========================>..] - ETA: 0s - loss: 1.4948 - acc: 0.4921
Epoch 00030: val_loss did not improve from 1.72165
97/97 [==============================] - 1s 11ms/step - loss: 1.4979 - acc: 0.4901 - val_loss: 2.0321 - val_acc: 0.3419
Epoch 31/400
93/97 [===========================>..] - ETA: 0s - loss: 1.4423 - acc: 0.5151
Epoch 00031: val_loss improved from 1.72165 to 1.69013, saving model to ../model_files/base_cnn_60_seconds_ppg_hr_wesad_ig.hdf5
97/97 [==============================] - 1s 11ms/step - loss

95/97 [============================>.] - ETA: 0s - loss: 0.6057 - acc: 0.8176
Epoch 00056: val_loss improved from 1.17830 to 1.09884, saving model to ../model_files/base_cnn_60_seconds_ppg_hr_wesad_ig.hdf5
97/97 [==============================] - 1s 12ms/step - loss: 0.6078 - acc: 0.8167 - val_loss: 1.0988 - val_acc: 0.6581
Epoch 57/400
96/97 [============================>.] - ETA: 0s - loss: 0.5837 - acc: 0.8255
Epoch 00057: val_loss did not improve from 1.09884
97/97 [==============================] - 1s 11ms/step - loss: 0.5849 - acc: 0.8250 - val_loss: 1.3473 - val_acc: 0.5897
Epoch 58/400
96/97 [============================>.] - ETA: 0s - loss: 0.5595 - acc: 0.8342
Epoch 00058: val_loss did not improve from 1.09884
97/97 [==============================] - 1s 10ms/step - loss: 0.5598 - acc: 0.8339 - val_loss: 1.1955 - val_acc: 0.6460
Epoch 59/400
93/97 [===========================>..] - ETA: 0s - loss: 0.5408 - acc: 0.8422
Epoch 00059: val_loss did not improve from 1.09884
97/97 [=

92/97 [===========================>..] - ETA: 0s - loss: 0.2557 - acc: 0.9250
Epoch 00085: val_loss did not improve from 0.85763
97/97 [==============================] - 1s 11ms/step - loss: 0.2569 - acc: 0.9246 - val_loss: 0.9451 - val_acc: 0.7266
Epoch 86/400
93/97 [===========================>..] - ETA: 0s - loss: 0.2579 - acc: 0.9230
Epoch 00086: val_loss improved from 0.85763 to 0.85154, saving model to ../model_files/base_cnn_60_seconds_ppg_hr_wesad_ig.hdf5
97/97 [==============================] - 1s 11ms/step - loss: 0.2584 - acc: 0.9230 - val_loss: 0.8515 - val_acc: 0.7559
Epoch 87/400
95/97 [============================>.] - ETA: 0s - loss: 0.2407 - acc: 0.9298
Epoch 00087: val_loss improved from 0.85154 to 0.81124, saving model to ../model_files/base_cnn_60_seconds_ppg_hr_wesad_ig.hdf5
97/97 [==============================] - 1s 12ms/step - loss: 0.2407 - acc: 0.9298 - val_loss: 0.8112 - val_acc: 0.7667
Epoch 88/400
93/97 [===========================>..] - ETA: 0s - loss: 0.2

Epoch 115/400
97/97 [==============================] - ETA: 0s - loss: 0.1520 - acc: 0.9547
Epoch 00115: val_loss did not improve from 0.72180
97/97 [==============================] - 1s 10ms/step - loss: 0.1520 - acc: 0.9547 - val_loss: 0.7288 - val_acc: 0.8025
Epoch 116/400
97/97 [==============================] - ETA: 0s - loss: 0.1512 - acc: 0.9549
Epoch 00116: val_loss did not improve from 0.72180
97/97 [==============================] - 1s 11ms/step - loss: 0.1512 - acc: 0.9549 - val_loss: 0.7285 - val_acc: 0.8076
Epoch 117/400
95/97 [============================>.] - ETA: 0s - loss: 0.1472 - acc: 0.9571
Epoch 00117: val_loss did not improve from 0.72180
97/97 [==============================] - 1s 10ms/step - loss: 0.1475 - acc: 0.9570 - val_loss: 0.7576 - val_acc: 0.8011
Epoch 118/400
94/97 [============================>.] - ETA: 0s - loss: 0.1501 - acc: 0.9548
Epoch 00118: val_loss did not improve from 0.72180
97/97 [==============================] - 1s 10ms/step - loss: 0.1507

Epoch 145/400
96/97 [============================>.] - ETA: 0s - loss: 0.1061 - acc: 0.9679
Epoch 00145: val_loss did not improve from 0.64821
97/97 [==============================] - 1s 10ms/step - loss: 0.1061 - acc: 0.9681 - val_loss: 0.7108 - val_acc: 0.8146
Epoch 146/400
97/97 [==============================] - ETA: 0s - loss: 0.1001 - acc: 0.9696
Epoch 00146: val_loss did not improve from 0.64821
97/97 [==============================] - 1s 10ms/step - loss: 0.1001 - acc: 0.9696 - val_loss: 0.8692 - val_acc: 0.7904
Epoch 147/400
94/97 [============================>.] - ETA: 0s - loss: 0.1038 - acc: 0.9696
Epoch 00147: val_loss did not improve from 0.64821
97/97 [==============================] - 1s 11ms/step - loss: 0.1034 - acc: 0.9697 - val_loss: 0.7101 - val_acc: 0.8151
Epoch 148/400
97/97 [==============================] - ETA: 0s - loss: 0.1049 - acc: 0.9687
Epoch 00148: val_loss did not improve from 0.64821
97/97 [==============================] - 1s 10ms/step - loss: 0.1049

95/97 [============================>.] - ETA: 0s - loss: 0.0894 - acc: 0.9732
Epoch 00175: val_loss did not improve from 0.60873
97/97 [==============================] - 1s 11ms/step - loss: 0.0888 - acc: 0.9734 - val_loss: 0.6838 - val_acc: 0.8309
Epoch 176/400
97/97 [==============================] - ETA: 0s - loss: 0.0775 - acc: 0.9759
Epoch 00176: val_loss did not improve from 0.60873
97/97 [==============================] - 1s 11ms/step - loss: 0.0775 - acc: 0.9759 - val_loss: 0.6357 - val_acc: 0.8393
Epoch 177/400
95/97 [============================>.] - ETA: 0s - loss: 0.0729 - acc: 0.9785
Epoch 00177: val_loss did not improve from 0.60873
97/97 [==============================] - 1s 11ms/step - loss: 0.0730 - acc: 0.9785 - val_loss: 0.6774 - val_acc: 0.8323
Epoch 178/400
94/97 [============================>.] - ETA: 0s - loss: 0.0713 - acc: 0.9787
Epoch 00178: val_loss did not improve from 0.60873
97/97 [==============================] - 1s 11ms/step - loss: 0.0716 - acc: 0.9788

Epoch 206/400
94/97 [============================>.] - ETA: 0s - loss: 0.0656 - acc: 0.9810
Epoch 00206: val_loss did not improve from 0.57764
97/97 [==============================] - 1s 11ms/step - loss: 0.0658 - acc: 0.9809 - val_loss: 0.7180 - val_acc: 0.8244
Epoch 207/400
95/97 [============================>.] - ETA: 0s - loss: 0.0608 - acc: 0.9802
Epoch 00207: val_loss did not improve from 0.57764
97/97 [==============================] - 1s 11ms/step - loss: 0.0605 - acc: 0.9802 - val_loss: 0.5943 - val_acc: 0.8524
Epoch 208/400
94/97 [============================>.] - ETA: 0s - loss: 0.0672 - acc: 0.9794
Epoch 00208: val_loss did not improve from 0.57764
97/97 [==============================] - 1s 11ms/step - loss: 0.0669 - acc: 0.9796 - val_loss: 0.6241 - val_acc: 0.8482
Epoch 209/400
96/97 [============================>.] - ETA: 0s - loss: 0.0657 - acc: 0.9797
Epoch 00209: val_loss did not improve from 0.57764
97/97 [==============================] - 1s 11ms/step - loss: 0.0658

Epoch 237/400
97/97 [==============================] - ETA: 0s - loss: 0.0536 - acc: 0.9838
Epoch 00237: val_loss did not improve from 0.51998
97/97 [==============================] - 1s 11ms/step - loss: 0.0536 - acc: 0.9838 - val_loss: 0.5833 - val_acc: 0.8579
Epoch 238/400
94/97 [============================>.] - ETA: 0s - loss: 0.0519 - acc: 0.9843
Epoch 00238: val_loss did not improve from 0.51998
97/97 [==============================] - 1s 10ms/step - loss: 0.0520 - acc: 0.9842 - val_loss: 0.5900 - val_acc: 0.8505
Epoch 239/400
94/97 [============================>.] - ETA: 0s - loss: 0.0511 - acc: 0.9848
Epoch 00239: val_loss did not improve from 0.51998
97/97 [==============================] - 1s 10ms/step - loss: 0.0510 - acc: 0.9849 - val_loss: 0.6386 - val_acc: 0.8463
Epoch 240/400
95/97 [============================>.] - ETA: 0s - loss: 0.0567 - acc: 0.9825
Epoch 00240: val_loss did not improve from 0.51998
97/97 [==============================] - 1s 10ms/step - loss: 0.0565

In [4]:
import tensorflow as tf
tf.autograph.set_verbosity(0)
import numpy as np
import os
import pandas as pd
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, Conv1D, GlobalMaxPooling1D, Dropout
from tensorflow.keras.utils import to_categorical
from alibi.explainers import IntegratedGradients
import matplotlib.pyplot as plt

In [11]:
n_steps = 50
method = "gausslegendre"
internal_batch_size = 100
nb_samples = 10
ig  = IntegratedGradients(model,
                          layer=model.layers[0],
                          n_steps=n_steps,
                          method=method,
                          internal_batch_size=internal_batch_size)

x_test_sample = test_x[:nb_samples]
predictions = model(x_test_sample).numpy().argmax(axis=1)

In [12]:
explanation = ig.explain(x_test_sample,
                         baselines=None,
                         target=predictions)

In [16]:
attrs = explanation.attributions
print('Attributions shape:', attrs.shape)

Attributions shape: (10, 30, 1)


In [17]:
attrs = attrs.sum(axis=2)
print('Attributions shape:', attrs.shape)

Attributions shape: (10, 30)


In [ ]:
i = 3
x_i = x_test_sample[i]
attrs_i = attrs[i]
pred = predictions[i]
pred_dict = {1: 'Positive review', 0: 'Negative review'}

print('Predicted label =  {}: {}'.format(pred, pred_dict[pred]))

from IPython.display import HTML
def  hlstr(string, color='white'):
    """
    Return HTML markup highlighting text with the desired color.
    """
    return f"<mark style=background-color:{color}>{string} </mark>"

def colorize(attrs, cmap='PiYG'):
    """
    Compute hex colors based on the attributions for a single instance.
    Uses a diverging colorscale by default and normalizes and scales
    the colormap so that colors are consistent with the attributions.
    """
    import matplotlib as mpl
    cmap_bound = np.abs(attrs).max()
    norm = mpl.colors.Normalize(vmin=-cmap_bound, vmax=cmap_bound)
    cmap = mpl.cm.get_cmap(cmap)

    # now compute hex values of colors
    colors = list(map(lambda x: mpl.colors.rgb2hex(cmap(norm(x))), attrs))
    return colors

words = decode_sentence(x_i, reverse_index).split()
colors = colorize(attrs_i)

HTML("".join(list(map(hlstr, words, colors))))